In [1]:
!pip install mitsuba

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 49.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import io
import tempfile
import os

import drjit as dr
import mitsuba as mi
print(mi.variants())
mi.set_variant("scalar_rgb")

['scalar_rgb', 'scalar_spectral', 'scalar_spectral_polarized', 'llvm_ad_rgb', 'llvm_ad_mono', 'llvm_ad_mono_polarized', 'llvm_ad_spectral', 'llvm_ad_spectral_polarized', 'cuda_ad_rgb', 'cuda_ad_mono', 'cuda_ad_mono_polarized', 'cuda_ad_spectral', 'cuda_ad_spectral_polarized']


In [3]:
n_rays    = 20
theta_max = np.deg2rad(20.0)

*Create mesh*

In [4]:
vertices = np.array([
    [-1.0, -1.0, 0.0],
    [ 1.0, -1.0, 0.0],
    [ 1.0,  1.0, 0.0],
    [-1.0,  1.0, 0.0],
], dtype=np.float32)

faces = np.array([
    [0, 1, 2],
    [0, 2, 3]
], dtype=np.int32)

*Create PLY string to be saved*

In [5]:
def create_ply_content(vertices, faces):
    """Create the content of a PLY file in ASCII format."""
    ply_content = io.StringIO()

    # Header PLY
    ply_content.write("ply\n")
    ply_content.write("format ascii 1.0\n")
    ply_content.write(f"element vertex {len(vertices)}\n")
    ply_content.write("property float x\n")
    ply_content.write("property float y\n")
    ply_content.write("property float z\n")
    ply_content.write(f"element face {len(faces)}\n")
    ply_content.write("property list uchar int vertex_index\n")
    ply_content.write("end_header\n")

    # Vertices
    for v in vertices:
        ply_content.write(f"{v[0]} {v[1]} {v[2]}\n")

    # Faces (triangles)
    for f in faces:
        ply_content.write(f"3 {f[0]} {f[1]} {f[2]}\n")

    return ply_content.getvalue()

# Generate the PLY string
ply_content = create_ply_content(vertices, faces)

*Write a file with the PLY mesh*

In [6]:
with tempfile.NamedTemporaryFile(mode='w', suffix='.ply', delete=False) as f:
    f.write(ply_content)
    temp_file_path = f.name

*Create the scene*

In [7]:
scene_dict = {
    "type": "scene",
    "square": {
        "type": "ply",
        "filename": temp_file_path
    }
}

scene = mi.load_dict(scene_dict)

*Ray origin*

In [8]:
origin = mi.Point3f(0.0, 0.0, 2.0)

*Angular (theta, phi) sampling for ray launching*

In [9]:
def sample_cone_directions(n_rays, theta_max):
    dirs = []
    for _ in range(n_rays):
        # Generate two uniform random numbers in [0, 1]
        # u1: controls the polar angle θ (distance from cone axis)
        # u2: controls the azimuthal angle φ (rotation around axis)
        u1, u2 = np.random.rand(), np.random.rand()

        # ==============================================================
        # POLAR ANGLE SAMPLING (θ) - CRITICAL STEP
        # ==============================================================
        # To sample directions uniformly within a cone of half-angle theta_max:
        #
        # 1. GOAL: Uniform distribution over solid angle (dΩ = sinθ dθ dφ)
        #    Probability density function (PDF) should be: p(θ,φ) = 1/(2π(1-cosθ_max))
        #    This is constant over the cone's solid angle.
        #
        # 2. CUMULATIVE DISTRIBUTION FUNCTION (CDF) FOR θ:
        #    The marginal CDF for θ is: F(θ) = (1 - cosθ) / (1 - cosθ_max)
        #    This comes from integrating the PDF over φ from 0 to 2π:
        #    ∫p(θ,φ) dφ = 2π/(2π(1-cosθ_max)) * sinθ = sinθ/(1-cosθ_max)
        #    Then integrating from 0 to θ: F(θ) = ∫₀^θ sinθ'/(1-cosθ_max) dθ'
        #                                     = (1 - cosθ)/(1 - cosθ_max)
        #
        # 3. INVERSE TRANSFORM SAMPLING:
        #    We set u1 = F(θ) and solve for θ:
        #    u1 = (1 - cosθ)/(1 - cosθ_max)
        #    => 1 - cosθ = u1 * (1 - cosθ_max)
        #    => cosθ = 1 - u1 * (1 - cosθ_max)
        #
        # 4. RESULT: This transformation ensures:
        #    - cosθ is uniformly distributed in [cosθ_max, 1]
        #    - θ is non-uniformly distributed in [0, θ_max]
        #    - Directions are uniformly distributed over the cone's solid angle
        cos_theta = 1.0 - u1 * (1.0 - np.cos(theta_max))

        # Compute sinθ from cosθ using identity sin²θ + cos²θ = 1
        sin_theta = np.sqrt(1.0 - cos_theta**2)

        # ==============================================================
        # AZIMUTHAL ANGLE SAMPLING (φ) - SIMPLE UNIFORM
        # ==============================================================
        # φ is uniformly distributed in [0, 2π]
        # This gives uniform rotation around the cone axis
        phi = 2.0 * np.pi * u2

        # ==============================================================
        # SPHERICAL TO CARTESIAN CONVERSION
        # ==============================================================
        # Convert spherical coordinates (θ,φ) to Cartesian (x,y,z):
        # x = sinθ * cosφ   (projection onto x-axis)
        # y = sinθ * sinφ   (projection onto y-axis)
        # z = -cosθ         (negative for downward-pointing cone)
        #
        # Note: z is negative because we want the cone to point downward
        # (toward negative z-axis). For upward cone, use z = cosθ.
        x = sin_theta * np.cos(phi)
        y = sin_theta * np.sin(phi)
        z = -cos_theta

        # Add the direction vector to the list
        dirs.append(mi.Vector3f(x, y, z))
    return dirs

directions = sample_cone_directions(n_rays, theta_max)

*Ray tracing*

In [10]:
for i, d in enumerate(directions):

    # Normalize the ray direction
    ray_direction = dr.normalize(d)

    # Create ray with origin and direction only
    ray = mi.Ray3f(o=origin, d=ray_direction)

    # Set maxt properties
    ray.maxt = mi.Float(np.inf)

    si = scene.ray_intersect(ray)

    print(f"\nRay {i}")

    if not si.is_valid():
        print("  No intersection")
        continue

    hit_point = si.p
    normal = dr.normalize(si.n)  # Create a copy of the normal

    print("  HIT")
    print(f"  Hit point : {hit_point}")
    print(f"  Normal    : {normal}")

    # Specular reflection
    d_in = ray_direction
    d_reflect = d_in - 2.0 * dr.dot(d_in, normal) * normal

    # Normalize reflection direction
    d_reflect = dr.normalize(d_reflect)

    # Create reflected ray
    reflected_ray = mi.Ray3f(
        o=hit_point + 1e-4 * normal,
        d=d_reflect
    )

    # Set maxt for reflected ray
    reflected_ray.maxt = mi.Float(np.inf)

    print(f"  Reflected direction: {reflected_ray.d}")



Ray 0
  HIT
  Hit point : [-0.0799816, 0.216863, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [-0.0397264, 0.107715, 0.993388]

Ray 1
  HIT
  Hit point : [0.0461185, -0.0503636, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [0.0230459, -0.0251671, 0.999418]

Ray 2
  HIT
  Hit point : [-0.0622402, -0.408366, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [-0.0304768, -0.199963, 0.979329]

Ray 3
  HIT
  Hit point : [-0.315951, -0.313041, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [-0.154208, -0.152788, 0.976153]

Ray 4
  HIT
  Hit point : [-0.116684, 0.281236, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [-0.0576775, 0.139016, 0.988609]

Ray 5
  HIT
  Hit point : [0.282099, 0.0505388, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [0.139623, 0.0250139, 0.989889]

Ray 6
  HIT
  Hit point : [-0.373356, -0.125014, 0]
  Normal    : [0, 0, 1]
  Reflected direction: [-0.183163, -0.0613297, 0.981168]

Ray 7
  HIT
  Hit point : [-0.0233395, 0.0489408, 0]
  Normal    :

*Remove temporary file*

In [11]:
os.unlink(temp_file_path)